In [1]:
import json
import chardet
import pandas as pd
import os


"""23년 vds 데이터 이상치 제거 """

folder_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\프로젝트\2025년\3. 도로 급경사 구간 교통정체를 고려한 적정 도로용량 산정 방안 연구 용역\시공도\vds 데이터\원본\20~24년\11월\22"
save_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\급경사\시공도\vds 데이터\이상치 제거\20~24년\11월\22"

json_path = r"C:\Workspace\vissimTest_Python\network_version2\resource\road_point.json"

output_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)
print(output_files)

# 노선명 json 파일 읽어옴
with open(json_path, "r", encoding="utf-8") as f:
    road_point = json.load(f)


for file_name in output_files:
    file_path = os.path.join(folder_path, file_name)

    # CSV 파일 인코딩 감지
    with open(file_path, "rb") as f:
        enc = chardet.detect(f.read())["encoding"]
        print(file_name, "=>", enc)

    df = pd.read_csv(file_path, encoding=enc)

    # 컬럼명 재정의를 통해 파일 포맷 일치화
    df.columns = ["date", "hour", "vds_id", "road_type_cd", "vmtc", "share", "avgspeed", "normal_cnt", "revision_cnt", "last_change"]

    # 불필요 컬럼 제거
    df = df.drop(columns=["date", "road_type_cd", "vmtc", "share", "normal_cnt", "revision_cnt", "last_change"])

    display("df : ", df)

    df["vds"] = df["vds_id"].str.extract(r'(\d{4})')

    # vds 이정 정보가 road_point에 있을 경우에만 남김
    df = df[df["vds_id"].isin(road_point.keys())]

    # vds 지점이정 매핑
    df["point"] = df["vds_id"].map(road_point)

    # 예시: 속도값이 -1~10인 행 제거
    if "avgspeed" in df.columns:
        df = df[~df["avgspeed"].between(-1, 10)]

    excel_file_path = os.path.join(save_path, file_name)

    #df.to_csv(excel_file_path, index=False)

    #print(f"{file_name} 생성 완료")


['TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221101.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221102.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221103.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221104.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221106.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221107.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221108.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221109.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221110.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221111.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221112.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221113.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221114.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221115.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221116.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221117.csv', 'TR021900070005-EX_MIN15_VDS_RTP_TPSTS_TB-20221118.csv', 'TR021900070005-EX_MIN15_VDS_R

'df : '

,hour,vds_id,avgspeed
0,0,0010VDE00100,84.85
1,0,0010VDE00200,97.29
2,0,0010VDE00300,89.88
3,0,0010VDE00400,83.72
4,0,0010VDE00500,92.55
...,...,...,...
754483,2345,6000VDS03150,-1.00
754484,2345,6000VDS03200,107.07
754485,2345,6000VDS03300,105.07
754486,2345,6000VDS03400,99.71


KeyboardInterrupt: 